In [59]:
### Library Import Initialization
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy.integrate import quad
import warnings
warnings.filterwarnings("ignore")


In [60]:
### Load in Stock Data
def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    return data

tickers = 'GOOG'
start_data = '2022-01-01'
stock_data = import_stock_data(tickers, start_data)
# Get most recent stock price
S_0 = stock_data.iloc[1, -1]
S_0

# Set the Strike Price (K) based on the current asset price (S_0)
K = 150

[*********************100%%**********************]  1 of 1 completed


In [61]:
### Compute Log Returns
def get_log_returns(data):
    return np.log(stock_data / stock_data.shift(1))

log_returns = get_log_returns(stock_data)
log_returns.iloc[-1]

GOOG    0.007203
Name: 2024-06-06 00:00:00, dtype: float64

In [62]:
### Calculate Historical Volatility (sigma)
def volatility(log_returns):
    # Calculate standard deviation of log_returns
    std_dev = log_returns.std()
    # Calculate historical volatility (v_0) = sqrt(252) * std dev
    sigma = np.sqrt(252) * std_dev # 252 trading days per year

    return sigma

sigma = volatility(log_returns)
print(sigma)

### Compute the variance (v_0) = sigma^2
v_0 = sigma ** 2
print(v_0)


GOOG    0.340401
dtype: float64
GOOG    0.115873
dtype: float64


In [63]:
### Coumpute the Characteristic Function
def char_func(u, S_0, r, T, k, theta, sigma, rho, v_0):
    i = 1j
    b = k - rho * sigma * i * u
    d = np.sqrt(b ** 2 + (sigma ** 2) * (i * u + u ** 2))
    g = (b - d) / (b + d)
    e_dt = np.exp(-d * T)
    C = r * i * u * T + (k * theta / (sigma ** 2)) * ((b - d) * T - 2 * np.log((1 - g * e_dt) / (1 - g)))
    D = ((b - d) / (sigma ** 2)) * ((1 - e_dt) / (1 - g * e_dt))
    
    char_eqn = np.exp(C + D * v_0 + i * u * np.log(S_0))

    return char_eqn

In [64]:
# Compute Probability P_1
def p_1(S0, K, T, r, kappa, theta, sigma, rho, v0):
    def integrand(u):
        numerator = np.exp(-1j * u * np.log(K)) * char_func(u - 1j, S0, r, T, kappa, theta, sigma, rho, v0)
        denominator = 1j * u * char_func(-1j, S0, r, T, kappa, theta, sigma, rho, v0)
        return np.real((numerator / denominator) * np.conj(numerator / denominator))

    result, error = quad(integrand, 0, np.inf)
    return 0.5 + (1 / np.pi) * result

# Compute Probability P_2
def p_2(S0, K, T, r, kappa, theta, sigma, rho, v0):
    def integrand(u):
        numerator = np.exp(-1j * u * np.log(K)) * char_func(u, S0, r, T, kappa, theta, sigma, rho, v0)
        denominator = 1j * u
        return np.real((numerator / denominator) * np.conj(numerator / denominator))

    result, error = quad(integrand, 0, np.inf)
    return 0.5 + (1 / np.pi) * result

In [65]:
### Compute the Heston Approximation for a Call and Put Option
def heston_approx(S_0, K, T, r, k, theta, sigma, rho, v_0, option_type):
    # Initialize Probabilities
    P_1 = p_1(S_0, K, T, r, k, theta, sigma, rho, v_0)
    P_2 = p_2(S_0, K, T, r, k, theta, sigma, rho, v_0)
    # Initialize Call Option Price
    call_price = (S_0 * P_1) - (K * np.exp(-r * T) * P_2)
    # Return Call Approx
    if option_type == 'call':
        return call_price
    # Return Put Approx using Put-Call Parity -> P - C = Ke^(-rt) - S_0 -> P = C + Ke^(-rt) - S_0
    if option_type == 'put':
        put_price = call_price + (K * np.exp(-r * T)) - S_0
        return put_price


In [66]:
### Initialize Parameters
# Kappa = Mean-reversion speed in the Heston model
k = 2 
# Long-term average variance (squared volatility) in the Heston model
theta = v_0 ** 2  
# rho - correlation parameter
rho = -0.5
# Time to Expiration (years)
T = 1
# Interest rate
r = 0.05

### Function Call
option_price = heston_approx(S_0, K, T, r, k, theta, sigma, rho, v_0, option_type = 'call')
print("Option Price Estimate:", option_price)

Option Price Estimate: 1.576466610014828
